# Spiculation Binary Classifier

## Prologue

### Import Libraries and paths to files.

In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
import pickle
from glob import glob
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import tensorflow as tf
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import datetime

# !pip -q install livelossplot 
# from livelossplot.tf_keras import PlotLossesCallback

In [ ]:
df_path = '/content/drive/MyDrive/Colab Notebooks/Research/parsedXMLnew.xlsx'
ds_path = '/content/nodules'
ds_file_1 = '/content/drive/MyDrive/Colab Notebooks/Research/dataset_before_bin.pickle'
ds_file_2 = '/content/drive/MyDrive/Colab Notebooks/Research/dataset_3_class.pickle'
ds_file_3 = '/content/drive/MyDrive/Colab Notebooks/Research/dataset_3_augmented.pickle'
# ds_file_grad = '/content/drive/MyDrive/Colab Notebooks/Research/dataset_grad.pickle'
save_path = '/content/drive/MyDrive/Colab Notebooks/Research/'

### Functions

#### Global

In [ ]:
def plot_heatmap(y_true, y_pred, class_names, ax, title):
  cm = confusion_matrix(y_true, y_pred)
  sns.heatmap(
      cm, 
      annot=True, 
      square=True, 
      xticklabels=class_names, 
      yticklabels=class_names,
      fmt='d', 
      cmap=plt.cm.Blues,
      cbar=False,
      ax=ax
  )
  ax.set_title(title, fontsize=16)
  ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")
  ax.set_ylabel('True Label', fontsize=12)
  ax.set_xlabel('Predicted Label', fontsize=12)

def disp_and_save_cam(img, gradcam, name, conv_layer_name):
    save_path = '/content/drive/MyDrive/Colab Notebooks/Research/visuals/'

    fig = plt.figure(figsize=(10,5))
    fig.add_subplot(1,3,1, title='Image with CAM')
    plt.imshow(img)
    plt.imshow(gradcam, alpha=0.5)
    plt.axis('off')

    fig.add_subplot(1,3,2, title='Image')
    plt.imshow(img)
    plt.imshow(img, alpha=0.5)
    plt.axis('off')

    fig.add_subplot(1,3,3, title='CAM')
    plt.imshow(gradcam)
    plt.axis('off')
    plt.savefig(f'{save_path}{conv_layer_name}/{name}.png')
    plt.show()
def gradcam(img, model, last_conv_layer_name, label, show=False, name=''):
  last_conv_layer = model.get_layer(last_conv_layer_name)
  last_conv_layer_model = tf.keras.Model(model.inputs, last_conv_layer.output)

  classifier_input = tf.keras.Input(shape=last_conv_layer.output.shape[1:])
  x = classifier_input 
  begin = False                
  for count, layer in enumerate(model.layers):
    if last_conv_layer_name == layer.name:
      begin = True
    if begin:
      x = layer(x)
      
  classifier_model = tf.keras.Model(classifier_input, x)

  with tf.GradientTape() as tape:
    inputs = img[np.newaxis, ...]
    last_conv_layer_output = last_conv_layer_model(inputs)
    tape.watch(last_conv_layer_output)
    pred = classifier_model(last_conv_layer_output)
    pred_class = 1 if tf.greater(pred, 0.5).numpy()[0,0] == True else 0

  grads = tape.gradient(pred, last_conv_layer_output)
  pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))


  last_conv_layer_output = last_conv_layer_output.numpy()[0]
  pooled_grads = pooled_grads.numpy()
  for i in range(pooled_grads.shape[-1]):
    last_conv_layer_output[:, :, i] *= pooled_grads[i]
  
  # Average over all the filters to get a single 2D array
  gradcam = np.mean(last_conv_layer_output, axis=-1)
  # Clip the values (equivalent to applying ReLU)
  # and then normalise the values
  gradcam = np.clip(gradcam, 0, np.max(gradcam)) / np.max(gradcam)
  gradcam = cv2.resize(gradcam, img.shape[:2])

  if show:
    disp_and_save_cam(img, gradcam, name, last_conv_layer_name)

#### Model 1

In [ ]:
def create_model_1(input_shape, bin_act_func, optimizer, loss_func=''):
  # VGG16 convolutional layers
  conv_base = tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=input_shape)
  # Freeze all conv_layers
  for layer in conv_base.layers:
    layer.trainable = False

  # Custom FC layers
  top_model = tf.keras.Sequential(
      [
        tf.keras.layers.GlobalAveragePooling2D(name='GAP_layer'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(3, activation='softmax', name='output_layer')
      ]
  )
  # Concat both models into one.
  model = tf.keras.Sequential()
  for layer in conv_base.layers:
    model.add(layer)
  model.add(top_model)
  model.compile(optimizer=optimizer, loss=loss_func, metrics=['accuracy'])
  return model

def fine_tune_model_1(num_layers, input_shape, bin_act_func, optimizer, loss_func, fname):
  # VGG16 convolutional layers
  conv_base = tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=input_shape)
  
  num_frozen = len(conv_base.layers)-num_layers-1
  # Freeze required conv_layers
  for layer in conv_base.layers[:num_frozen]:
    layer.trainable = False

  # Custom FC layers
  top_model = tf.keras.Sequential(
      [
        tf.keras.layers.GlobalAveragePooling2D(name='GAP_layer'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(3, activation='softmax', name='output_layer')
      ]
  )
  
  # Concat both models into one.
  model = tf.keras.Sequential()
  for layer in conv_base.layers:
    model.add(layer)
  model.add(top_model)
  model.load_weights(fname)
  model.compile(optimizer=optimizer, loss=loss_func, metrics=['accuracy'])
  return model

#### Model 2

In [ ]:
def create_model_2(input_shape, dp_rates, bin_act_func, optimizer, loss_func=''):
  # VGG16 convolutional layers
  conv_base = tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=input_shape)
  # Freeze all conv_layers
  for layer in conv_base.layers:
    layer.trainable = False

  # Custom FC layers
  top_model = tf.keras.Sequential(
      [
        tf.keras.layers.GlobalAveragePooling2D(name='GAP_layer'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(dp_rates[0]),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(dp_rates[1]),       
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(dp_rates[2]),
        tf.keras.layers.Dense(3, activation='softmax', name='output_layer')
      ]
  )
  # Concat both models into one.
  model = tf.keras.Sequential()
  for layer in conv_base.layers:
    model.add(layer)
  model.add(top_model)
  model.compile(optimizer=optimizer, loss=loss_func, metrics=['accuracy'])
  return model

def fine_tune_model_2(num_layers, input_shape, dp_rates, bin_act_func, optimizer, loss_func, fname):
  # VGG16 convolutional layers
  conv_base = tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=input_shape)
  
  num_frozen = len(conv_base.layers)-num_layers-1
  # Freeze required conv_layers
  for layer in conv_base.layers[:num_frozen]:
    layer.trainable = False

  # Custom FC layers
  top_model = tf.keras.Sequential(
      [        
        tf.keras.layers.GlobalAveragePooling2D(name='GAP_layer'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(dp_rates[0]),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(dp_rates[1]),       
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(dp_rates[2]),
        tf.keras.layers.Dense(3, activation='softmax', name='output_layer')
      ]
  )
  
  # Concat both models into one.
  model = tf.keras.Sequential()
  for layer in conv_base.layers:
    model.add(layer)
  model.add(top_model)
  model.load_weights(fname)
  model.compile(optimizer=optimizer, loss=loss_func, metrics=['accuracy'])
  return model

## Preprocessing



### Extract Labels and Binning Them

We only need the InstanceID (nodule id) and the Spiculation rating (label)

In [ ]:
# df = pd.read_excel(df_path)
# df = df[['InstanceID', 'spiculation']]
# df.head()

Load the images and match their corresponding labels.

In [ ]:
shapes = []
ds = []
if os.path.isfile(ds_file_1):
  with open(ds_file_1, 'rb') as f:
    ds = pickle.load(f)
  for d in ds:
    shapes.append(d[0].shape)
  shapes = np.array(shapes)
else:
  !tar xzf '/content/drive/MyDrive/Colab Notebooks/Research/LIDC/nodules.tar.gz'

  file_names = glob(os.path.join(ds_path,'*.png'))
  id_set = set(df['InstanceID'])
  for f in file_names:
    id = int(f.split('/')[-1].split('.')[0])
    # print('InstanceID:', id, end='\n\n')
    if id in id_set:
      label =  df[df['InstanceID']==id]['spiculation'].iloc[0]
      img = cv2.imread(f, 0)
      shapes.append(img.shape)
      ds.append([img, label, id])
  shapes = np.array(shapes)

  with open(os.path.join(save_path,'dataset_before_bin.pickle'), 'wb') as f:
    pickle.dump(ds, f)

Load dataset from pickle file

In [ ]:
fig = plt.figure(figsize=(10,5))
plt.title('Total Number of Images Before Binning')
labels, counts = np.unique([d[1] for d in ds], return_counts='True')
bars = plt.bar(labels, counts, align='center')
plt.gca().set_xticks(labels)
plt.ylabel('Frequency')
plt.xlabel('Spiculation Rating')
for bar in bars:
  yval = bar.get_height()
  plt.text(bar.get_x()+0.3, yval + 200, yval)
plt.savefig(os.path.join(save_path,'bar_before_bin'))
plt.show()

Spiculation rating is, 1 least spiculation, to, 5 most spiculation. As we can see 1 has the most frequency compared to 5.

In [ ]:
# df['spiculation'] = df['spiculation'].replace([2,4], [1,5])
# df['spiculation'] = df['spiculation'].replace([1,3,5], [0,1,2])
# df.head()

In [ ]:
if os.path.isfile(ds_file_2):
  with open(ds_file_2, 'rb') as f:
    ds = pickle.load(f)
else:
  for d in ds:
    d[1] =  df[df['InstanceID']==d[2]]['spiculation'].iloc[0]

  with open(os.path.join(save_path,'dataset_3_class.pickle'), 'wb') as f:
    pickle.dump(ds, f)

In [ ]:
fig = plt.figure(figsize=(10,5))
plt.title('Total Number of Images After Bining')
labels, counts = np.unique([d[1] for d in ds], return_counts='True')
bars = plt.bar(labels, counts, align='center')
plt.gca().set_xticks(labels)
plt.ylabel('Frequency')
plt.xlabel('Spiculation Rating')
for bar in bars:
  yval = bar.get_height()
  plt.text(bar.get_x()+0.3, yval + 200, yval)
plt.savefig(os.path.join(save_path,'bar_bin'))
plt.show()

### Scaling Images


In [ ]:
temp = []
input_size = (128,128,3)
min_size = (32,32)

for d in ds:
  if d[0].shape >= min_size:
    if d[0].shape < input_size[:2]:
      # Up-Scaling
      d[0] = cv2.resize(d[0], dsize=input_size[:2], interpolation=cv2.INTER_CUBIC)
    elif d[0].shape > input_size:
      # Down-Scaling
      d[0] = cv2.resize(d[0], dsize=input_size[:2], interpolation=cv2.INTER_AREA)
    temp.append(d)

del(ds)
ds = temp

In [ ]:
fig = plt.figure(figsize=(10,5))
plt.title(f'Total Number of Images After Bining and Images Greater than {min_size}')
labels, counts = np.unique([d[1] for d in ds], return_counts='True')
bars = plt.bar(labels, counts, align='center')
plt.gca().set_xticks(labels)
plt.ylabel('Frequency')
plt.xlabel('Spiculation Rating')
for bar in bars:
  yval = bar.get_height()
  plt.text(bar.get_x()+0.3, yval + 10, yval)
plt.savefig(os.path.join(save_path,'bar_bin'))
plt.show()

### Random Sampling

In [ ]:
class_0, class_1, class_2 = [], [], []
for d in ds:
  if d[1] == 0:
    class_0.append(d.copy())
  elif d[1] == 1:
    class_1.append(d.copy())
  else:
    class_2.append(d.copy())
del(ds)
bin_0 = []
bin_1 = class_1.copy()
bin_2 = []

# Indicies already sampled
used = set()
while len(bin_0) != len(bin_1):
  idx = np.random.randint(low=0, high=len(class_0))
  if idx not in used:
    bin_0.append(class_0[idx].copy())
    used.add(idx)

used = set()
while len(bin_2) != len(bin_1):
  idx = np.random.randint(low=0, high=len(class_2))
  if idx not in used:
    bin_2.append(class_2[idx].copy())
    used.add(idx)

del(class_0)
del(class_1)
del(class_2)

ds = bin_0.copy()
ds.extend(bin_1.copy())
ds.extend(bin_2.copy())

del(bin_0)
del(bin_1)
del(bin_2)

In [ ]:
fig = plt.figure(figsize=(10,5))
plt.title(f'Total Number of Images After Bining and Images Greater than {min_size}')
labels, counts = np.unique([d[1] for d in ds], return_counts='True')
bars = plt.bar(labels, counts, align='center')
plt.gca().set_xticks(labels)
plt.ylabel('Frequency')
plt.xlabel('Spiculation Rating')
for bar in bars:
  yval = bar.get_height()
  plt.text(bar.get_x()+0.3, yval + 10, yval)
plt.savefig(os.path.join(save_path,'bar_bin'))
plt.show()

### Grayscale -> RGB

In [ ]:
for d in ds:
  d[0] = np.dstack([d[0]]*3)

### Split Training and Test Sets
Now that we have the dataset ready. Its time to split it into training and test sets, as well as randomizing and converting to tensors.

In [ ]:
X, y = [], []
for d in ds:
  X.append(d[0])
  y.append(d[1])

X, y = np.array(X), tf.keras.utils.to_categorical(y, num_classes=3)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=1)

In [ ]:
fig = plt.figure(figsize=(10,5))
plt.title(f'Total Number of Images After Bining and Images Greater than {min_size}')
labels, counts = np.unique(y_train, return_counts='True')
bars = plt.bar(labels, counts, align='center')
plt.gca().set_xticks(labels)
plt.ylabel('Frequency')
plt.xlabel('Spiculation Rating')
for bar in bars:
  yval = bar.get_height()
  plt.text(bar.get_x()+0.3, yval + 10, yval)
plt.savefig(os.path.join(save_path,'bar_bin'))
plt.show()

### Data Augmentation

In [ ]:
ImgGen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=360, shear_range=10, horizontal_flip=True, brightness_range=[0.5,1.5], vertical_flip=True, width_shift_range=.1, height_shift_range=.1, validation_split=.2, dtype=tf.uint8)
train_generator = ImgGen.flow(X_train, y_train, batch_size=64, subset='training', seed=1)
val_generator = ImgGen.flow(X_train, y_train, batch_size=64, subset='validation', seed=1)

## Model 1

In [ ]:
learning_rate = 0.0001
# batch_size = 64
n_epochs = 100
bin_act_func = 'softmax'
loss_func = 'categorical_crossentropy'
optim = tf.keras.optimizers.Adam(learning_rate=learning_rate)

### No Fine Tuning

In [ ]:
# fine_tune = 3
with tf.device('/GPU:0'):
  model_1 = create_model_1(input_size, bin_act_func, optim, loss_func)
model_1.summary()

In [ ]:
# CallBacks
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, mode='min')
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model_1_history = model_1.fit(train_generator, validation_data=val_generator, epochs=n_epochs, verbose=2, callbacks=[early_stop, tensorboard_callback])

In [ ]:
fname = os.path.join(save_path,f'weights/model_1_weights_no_fine_tuning.h5')
model_1.save_weights(fname)

In [ ]:
y_pred_1 = tf.argmax(model_1.predict(X_test), axis=1)

class_names = ['0','1']
fig, ax = plt.subplots(1, 1, figsize=(5, 5))

plot_heatmap(y_test, y_pred_1, class_names, ax, title="Confusion Matrix for Model A")    

fig.tight_layout()
fig.subplots_adjust(top=1.25)
plt.show()

In [ ]:
print(classification_report(y_test, y_pred_1, digits=3))

### Fine Tuning Last Layer

In [ ]:
with tf.device('/GPU:0'):
  model_1_ft1 = fine_tune_model_1(1,input_size, bin_act_func, optim, loss_func, fname)

In [ ]:
# CallBacks
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, mode='min')
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model_1_ft1_history = model_1_ft1.fit(train_generator, validation_data=val_generator, epochs=n_epochs, verbose=2, callbacks=[early_stop, tensorboard_callback])

In [ ]:
fname = os.path.join(save_path,f'weights/model_1_weights_fine_tuning_1.h5')
model_1_ft1.save_weights(fname)

In [ ]:
y_pred_1 = tf.greater(model_1_ft1.predict(X_test), 0.5)

class_names = ['0','1']
fig, ax = plt.subplots(1, 1, figsize=(5, 5))

plot_heatmap(y_test, y_pred_1, class_names, ax, title="Confusion Matrix for Model A")    

fig.tight_layout()
fig.subplots_adjust(top=1.25)
plt.show()

In [ ]:
print(classification_report(y_test, y_pred_1, digits=3))

### Fine Tuning Last 2 Layers

In [ ]:
with tf.device('/GPU:0'):
  model_1_ft2 = fine_tune_model_1(2,input_size, bin_act_func, optim, loss_func, fname)

In [ ]:
# CallBacks
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, mode='min')
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model_1_ft1_history = model_1_ft2.fit(train_generator, validation_data=val_generator, epochs=n_epochs, verbose=2, callbacks=[early_stop, tensorboard_callback])

In [ ]:
fname = os.path.join(save_path,f'weights/model_1_weights_fine_tuning_2.h5')
model_1_ft2.save_weights(fname)

In [ ]:
y_pred_1 = tf.greater(model_1_ft2.predict(X_test), 0.5)

class_names = ['0','1']
fig, ax = plt.subplots(1, 1, figsize=(5, 5))

plot_heatmap(y_test, y_pred_1, class_names, ax, title="Confusion Matrix for Model A")    

fig.tight_layout()
fig.subplots_adjust(top=1.25)
plt.show()

In [ ]:
print(classification_report(y_test, y_pred_1, digits=3))

### Fine Tuning Last 3 Layers

In [ ]:
with tf.device('/GPU:0'):
  model_1_ft3 = fine_tune_model_1(3,input_size, bin_act_func, optim, loss_func, fname)

In [ ]:
# CallBacks
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, mode='min')
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model_1_ft3_history = model_1_ft3.fit(train_generator, validation_data=val_generator, epochs=n_epochs, verbose=2, callbacks=[early_stop, tensorboard_callback])

In [ ]:
fname = os.path.join(save_path,f'weights/model_1_weights_fine_tuning_3.h5')
model_1_ft3.save_weights(fname)

In [ ]:
y_pred_1 = tf.greater(model_1_ft3.predict(X_test), 0.5)

class_names = ['0','1']
fig, ax = plt.subplots(1, 1, figsize=(5, 5))

plot_heatmap(y_test, y_pred_1, class_names, ax, title="Confusion Matrix for Model A")    

fig.tight_layout()
fig.subplots_adjust(top=1.25)
plt.show()

In [ ]:
print(classification_report(y_test, y_pred_1, digits=3))

## Model 2

In [ ]:
learning_rate = 0.0001
n_epochs = 100
bin_act_func = 'sigmoid'
loss_func = 'categorical_crossentropy'
optim = tf.keras.optimizers.Adam(learning_rate=learning_rate)

### No Fine Tuning

In [ ]:
dp_rates = [.5,.5,.5]
with tf.device('/GPU:0'):
  model_2 = create_model_2(input_size, dp_rates, bin_act_func, optim, loss_func)

In [ ]:
# CallBacks
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, mode='min')
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model_2_history = model_2.fit(train_generator, validation_data=val_generator, epochs=n_epochs, verbose=2, callbacks=[early_stop, tensorboard_callback])

In [ ]:
fname = os.path.join(save_path,f'weights/model_2_weights_no_fine_tuning.h5')
model_2.save_weights(fname)

In [ ]:
y_pred_2 = tf.greater(model_2.predict(X_test), 0.5)

class_names = ['0','1']
fig, ax = plt.subplots(1, 1, figsize=(5, 5))

plot_heatmap(y_test, y_pred_2, class_names, ax, title="Confusion Matrix for Model A")    

fig.tight_layout()
fig.subplots_adjust(top=1.25)
plt.show()

In [ ]:
print(classification_report(y_test, y_pred_2, digits=3))

### Fine Tuning Last Layer

In [ ]:
with tf.device('/GPU:0'):
  model_2_ft1 = fine_tune_model_2(1,input_size, dp_rates, bin_act_func, optim, loss_func, fname)

In [ ]:
# CallBacks
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, mode='min')
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model_2_ft1_history = model_2_ft1.fit(train_generator, validation_data=val_generator, epochs=n_epochs, verbose=2, callbacks=[early_stop, tensorboard_callback])

In [ ]:
fname = os.path.join(save_path,f'weights/model_2_weights_fine_tuning_1.h5')
model_2_ft1.save_weights(fname)

In [ ]:
y_pred_2 = tf.greater(model_2_ft1.predict(X_test), 0.5)

class_names = ['0','1']
fig, ax = plt.subplots(1, 1, figsize=(5, 5))

plot_heatmap(y_test, y_pred_2, class_names, ax, title="Confusion Matrix for Model A")    

fig.tight_layout()
fig.subplots_adjust(top=1.25)
plt.show()

In [ ]:
print(classification_report(y_test, y_pred_2, digits=3))

### Fine Tuning Last 2 Layers


In [ ]:
with tf.device('/GPU:0'):
  model_2_ft2 = fine_tune_model_2(1,input_size, dp_rates, bin_act_func, optim, loss_func, fname)

In [ ]:
# CallBacks
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, mode='min')
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model_2_ft2_history = model_2_ft2.fit(train_generator, validation_data=val_generator, epochs=n_epochs, verbose=2, callbacks=[early_stop, tensorboard_callback])

In [ ]:
fname = os.path.join(save_path,f'weights/model_2_weights_fine_tuning_2.h5')
model_2_ft2.save_weights(fname)

In [ ]:
y_pred_2 = tf.greater(model_2_ft2.predict(X_test), 0.5)

class_names = ['0','1']
fig, ax = plt.subplots(1, 1, figsize=(5, 5))

plot_heatmap(y_test, y_pred_2, class_names, ax, title="Confusion Matrix for Model A")    

fig.tight_layout()
fig.subplots_adjust(top=1.25)
plt.show()

In [ ]:
print(classification_report(y_test, y_pred_2, digits=3))

### Fine Tuning Last 3 Layers

In [ ]:
with tf.device('/GPU:0'):
  model_2_ft3 = fine_tune_model_2(1,input_size, dp_rates, bin_act_func, optim, loss_func, fname)

In [ ]:
# CallBacks
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, mode='min')
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model_2_ft3_history = model_2_ft3.fit(train_generator, validation_data=val_generator, epochs=n_epochs, verbose=2, callbacks=[early_stop, tensorboard_callback])

In [ ]:
fname = os.path.join(save_path,f'weights/model_2_weights_fine_tuning_3.h5')
model_2_ft3.save_weights(fname)

In [ ]:
y_pred_2 = tf.greater(model_2_ft3.predict(X_test), 0.5)

class_names = ['0','1']
fig, ax = plt.subplots(1, 1, figsize=(5, 5))

plot_heatmap(y_test, y_pred_2, class_names, ax, title="Confusion Matrix for Model A")    

fig.tight_layout()
fig.subplots_adjust(top=1.25)
plt.show()

In [ ]:
print(classification_report(y_test, y_pred_2, digits=3))

## TensorBoard

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
# Clear any logs from previous runs
# %rm -rf ./logs/
%tensorboard --logdir logs/fit

## Grad-CAM

In [ ]:
model = model_2_ft2

In [ ]:
last_conv_layer = 'block1_conv2'
# match = []
# indicies = []
# for idx,(img,label) in enumerate(zip(X_test,y_test)):
#   i = np.random.randint(0,10)
#   j = np.random.randint(0,10)
#   show = False
#   if i == j:
#     match.append((img, label))
#     indicies.append(idx)
#     show = True
#   gradcam(img, model, last_conv_layer, label, show=show, name=idx)

#   # guided_gradcam(img, model, last_conv_layer, np.argmax(label), show=show)
#   if len(match) == 10:
#     break
for idx, (img,label) in zip(indicies,match):
  gradcam(img, model, last_conv_layer, np.argmax(label), show=True, name=idx)

In [ ]:
last_conv_layer = 'block2_conv2'
for idx, (img,label) in zip(indicies,match):
  gradcam(img, model, last_conv_layer, np.argmax(label), show=True, name=idx)
  # guided_gradcam(img, model, last_conv_layer, np.argmax(label), show=True)

In [ ]:
last_conv_layer = 'block3_conv3'
for idx, (img,label) in zip(indicies,match):
  gradcam(img, model, last_conv_layer, np.argmax(label), show=True, name=idx)

In [ ]:
last_conv_layer = 'block4_conv3'
for idx, (img,label) in zip(indicies,match):
  gradcam(img, model, last_conv_layer, np.argmax(label), show=True, name=idx)

In [ ]:
last_conv_layer = 'block5_conv3'
for idx, (img,label) in zip(indicies,match):
  gradcam(img, model, last_conv_layer, np.argmax(label), show=True, name=idx)

In [ ]:
eval_list = []
for idx in indicies:
  eval_list.append((idx, y_test[idx],1 if y_pred_1[idx].numpy()[0] else 0))
np.savetxt(save_path+'12x12match.txt', eval_list)
eval_list